# Dash Python - Introduction

[Dash website](https://dash.plot.ly/)  
[Dash tutorial](https://dash.plot.ly/installation)

## Dash installation
Install the required packages
```bash
pip install dash==0.39.0  # The core dash backend
pip install dash-html-components==0.14.0  # HTML components
pip install dash-core-components==0.44.0  # Supercharged components
pip install dash-table==3.6.0  # Interactive DataTable component (new!)
pip install dash-daq==0.1.0  # DAQ components (newly open-sourced!)
``` 

## Dash layout
Create a python file `app.py`
### Load the required libraries
```python
import dash
import dash_core_components as dcc
import dash_html_components as html
```

Create the app object, you can pass an external stylesheet object to it

```python
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

if __name__ == '__main__':
    app.run_server(debug=True)
```
The option `debug=True` activates the "hot-reloading" feature.
has is Dash is going to monitor if you app is changed and immediately refresh your browser
to show changes on the client. 

Add a `layout` to the app object by using HTML and Supercharged components

```python
app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])
```

Then run the app.

```bash
$ python app.py
...Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
```

By default, it will listen on the `localhost` at port `8050`.

### Adding style to HTML components
```python
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Hello Dash',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(children='Dash: A web application framework for Python.', style={
        'textAlign': 'center',
        'color': colors['text']
    }),

    dcc.Graph(
        id='example-graph-2',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'plot_bgcolor': colors['background'],
                'paper_bgcolor': colors['background'],
                'font': {
                    'color': colors['text']
                }
            }
        }
    )
])

```

Remind that styles are:
- given as a `dictionary`  
- in camelCased form: use `textAlign` for "text-align"  

Use `className` property to refer to HTML `class`


## Adding Reusable Components

We start by reading some data
```python
import pandas as pd

df = pd.read_csv(
    'https://forge.scilab.org/index.php/p/rdataset/'
    'source/tree/master/csv/ggplot2/msleep.csv')
```

to write a function that given a dataset prints an HTML table
```python
def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )
```

and then we add it at the end of our layout
```python
    generate_table(df)
```


## A more sofisticated graph

Import the `plotly.graph_objs` library
```python
import plotly.graph_objs as go
```

and replace the previous existing `dcc.Graph` in our layout with the following one
```python
    dcc.Graph(
            id='life-exp-vs-gdp',
            figure={
                'data': [
                    go.Scatter(
                        x=df[df['continent'] == i]['gdp per capita'],
                        y=df[df['continent'] == i]['life expectancy'],
                        text=df[df['continent'] == i]['country'],
                        mode='markers',
                        opacity=0.7,
                        marker={
                            'size': 15,
                            'line': {'width': 0.5, 'color': 'white'}
                        },
                        name=i
                    ) for i in df.continent.unique()
                ],
                'layout': go.Layout(
                    xaxis={'type': 'log', 'title': 'GDP Per Capita'},
                    yaxis={'title': 'Life Expectancy'},
                    margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                    legend={'x': 0, 'y': 1},
                    hovermode='closest'
                )
            }
        )
```


## Adding some Markdown content

The `Markdown` component in the `dash_core_components` library
allows to simply include in the layout some markdown content.

Define the markdown test
```python
markdown_text = '''
### Dash and Markdown

Dash apps can be written in Markdown.
Dash uses the [CommonMark](http://commonmark.org/)
specification of Markdown.
Check out their [60 Second Markdown Tutorial](http://commonmark.org/help/)
if this is your first introduction to Markdown!
'''
```

and then include them at the beginning of the layout
by calling the `Markdown` component
```python
    dcc.Markdown(children=markdown_text)
```

By doing so remove the first two HTML components, 
`html.H1` and `html.div`,
together with the styles.


## Core Components

To see the set of widgets component included in the
`dash_core_components` library add the following `html.Div`
 at the end of the layout.
```python
    html.Div([
        html.Label('Dropdown'),
        dcc.Dropdown(
            options=[
                {'label': 'New York City', 'value': 'NYC'},
                {'label': u'Montréal', 'value': 'MTL'},
                {'label': 'San Francisco', 'value': 'SF'}
            ],
            value='MTL'
        ),
    
        html.Label('Multi-Select Dropdown'),
        dcc.Dropdown(
            options=[
                {'label': 'New York City', 'value': 'NYC'},
                {'label': u'Montréal', 'value': 'MTL'},
                {'label': 'San Francisco', 'value': 'SF'}
            ],
            value=['MTL', 'SF'],
            multi=True
        ),
    
        html.Label('Radio Items'),
        dcc.RadioItems(
            options=[
                {'label': 'New York City', 'value': 'NYC'},
                {'label': u'Montréal', 'value': 'MTL'},
                {'label': 'San Francisco', 'value': 'SF'}
            ],
            value='MTL'
        ),
    
        html.Label('Checkboxes'),
        dcc.Checklist(
            options=[
                {'label': 'New York City', 'value': 'NYC'},
                {'label': u'Montréal', 'value': 'MTL'},
                {'label': 'San Francisco', 'value': 'SF'}
            ],
            values=['MTL', 'SF']
        ),
    
        html.Label('Text Input'),
        dcc.Input(value='MTL', type='text'),
    
        html.Label('Slider'),
        dcc.Slider(
            min=0,
            max=9,
            marks={i: 'Label {}'.format(i) if i == 1 else str(i) for i in range(1, 6)},
            value=5,
        ),
    ], style={'columnCount': 2})
```

## Basic Dash Callbacks
Similarly to _R Shiny_ we need to comunicate objcts between the client and server and viceversa.
To do this we import `Input` and `Output` from `dash.dependencies` library.
### Load the required libraries
```python
from dash.dependencies import Input, Output
```


### Update the content of a Div
We give a identifier, `my-dropdown`, to the dropdown control
```python
    html.Label('Dropdown'),
            dcc.Dropdown(
                id='my-dropdown',
                options= opt_vore,
                value= df_vore[0]
            ),
```

and we add a `html.Div`, with identifier `my-div`, to the layout
before le list of widgets, substituting the table. We add to it some formitting styles.
```python
    html.Div(id='my-div',
             style={
                 'background' : 'yellow',
                 'color' : 'blue'
             }),
```

Finally we add then the call back function, after the definition of the `app.layout`
that receives the input from the Dropdown manu and update as output the condent of the Div.

```python
@app.callback(
    Output(component_id='my-div', component_property='children'),
    [Input(component_id='my-dropdown', component_property='value')]
)
def update_output_div(input_value):
    return 'You\'ve entered "{}"'.format(input_value)
```


### Update the content Graph 
First we copy  the content of the `dcc.Graph.figure` 
and return it from a new function that we call
`update_output_graph`.
We slightly modify the definition of `data`
```python
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    [Input(component_id='my-dropdown', component_property='value')]
)
def update_output_graph(input_value):
    return  {
                'data': [
                    go.Scatter(
                        x=df[df['vore'] == i]['bodywt'] if i == input_value else [],
                        y=df[df['vore'] == i]['sleep_total'] if i == input_value else [],
                        text=df[df['vore'] == i]['name'],
                        mode='markers',
                        opacity=0.7,
                        marker={
                            'size': 15,
                            'line': {'width': 0.5, 'color': 'white'}
                        },
                        name=i
                    ) for i in df_vore
                ],
                'layout': go.Layout(
                    xaxis={'type': 'log', 'title': 'Body weight (kg)'},
                    yaxis={'title': 'Total daily sleep time (hr)'},
                    margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                    legend={'x': 0, 'y': 1},
                    hovermode='closest'
                )
            }
```

and remove this information from the Graph component in the layuot,
making also sure that the `id` property is properly set.
```python
    dcc.Graph(id='my-graph'),
```


### Using the Multiple selection

Let us remove the Dropdown, the`html.Div` and
add the identifier `id='my-multi-dropdown'` to the Multi-Select Dropdown.

Then update the function `update_output_graph` as in the following
```python
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    [Input(component_id='my-multi-dropdown', component_property='value')]
)
def update_output_graph(input_value):
    return  {
                'data': [
                    go.Scatter(
                        x=df[df['vore'] == i]['bodywt'] if i in input_value else [],
                        y=df[df['vore'] == i]['sleep_total'] if i in input_value else [],
                        text=df[df['vore'] == i]['name'],
                        mode='markers',
                        opacity=0.7,
                        marker={
                            'size': 15,
                            'line': {'width': 0.5, 'color': 'white'}
                        },
                        name=i
                    ) for i in df_vore
                ],
                'layout': go.Layout(
                    xaxis={'type': 'log', 'title': 'Body weight (kg)'},
                    yaxis={'title': 'Total daily sleep time (hr)'},
                    margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                    legend={'x': 0, 'y': 1},
                    hovermode='closest'
                )
            }
```


### Using Multiple Outputs

We now add a new graph that shows the boxplot of the total sleep time for the selected type of mammals.

A list of possible chart available is accessible in the following link
[Statistical Charts](https://plot.ly/python/statistical-charts/)

We first add a `dcc.Graph` in the layout after the one with `id='my-graph'`
```python
    dcc.Graph(id='my-box-plot'),
```

Then we update the function `update_output_graph`to add the data to the graph
`my-box-plot`. Here it is the complete function modified
```python
@app.callback(
    [Output('my-graph', 'figure'),
     Output('my-box-plot', 'figure'),],
    [Input('my-multi-dropdown', 'value')]
)
def update_output_graph(input_value):
    return  {
                'data': [
                    go.Scatter(
                        x=df[df['vore'] == i]['bodywt'] if i in input_value else [],
                        y=df[df['vore'] == i]['sleep_total'] if i in input_value else [],
                        text=df[df['vore'] == i]['name'],
                        mode='markers',
                        opacity=0.7,
                        marker={
                            'size': 15,
                            'line': {'width': 0.5, 'color': 'white'}
                        },
                        name=i
                    ) for i in df_vore
                ],
                'layout': go.Layout(
                    xaxis={'type': 'log', 'title': 'Body weight (kg)'},
                    yaxis={'title': 'Total daily sleep time (hr)'},
                    margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                    legend={'x': 0, 'y': 1},
                    hovermode='closest'
                )
            },            {
                'data': [ go.Box(
                            y= df[df['vore'] == i]['sleep_total'],
                            name= i + 'vore'
                        ) if i in input_value else []
                          for i in df_vore ]
            }
```


### Chained Callbacks

We remove all input widget besides the Slider that we call
`my-slider`, and we modify like this

```python
        html.Label('Slider'),
        html.Div(
            dcc.RangeSlider(
                id='my-slider',
                step= 0.1
            ),
            style={
                'margin': '10%'
            }
        ),
```

Then we create a callback function to handle its attributes
```python
@app.callback(
    [Output('my-slider', 'min'), Output('my-slider', 'max'), Output('my-slider', 'value'), Output('my-slider', 'marks')],
    [Input('my-multi-dropdown', 'value')]
)
def update_slider(input_value):
    def round(x):
        return int(x) if x % 0.1 < 0.1 else x
    data = df[df.vore.isin(input_value)]['sleep_total']
    min = round(data.min())
    max = round(data.max())
    mean = round(data.mean())
    low = round((min + mean)/2)
    high = round((max + mean) / 2)
    marks = {min: {'label': min, 'style': {'color': '#77b0b1'}},
             max: {'label': max, 'style': {'color': '#77b0b1'}}}
    return min, max,  [low, high], marks
```

and finally we filter the `my-graph` data according to the Slider Range.
```python
@app.callback(
    [Output('my-graph', 'figure'),
     Output('my-box-plot', 'figure'),],
    [Input('my-multi-dropdown', 'value'), Input('my-slider', 'value')]
)
def update_output_graph(input_value, slider_range):
    if (len(slider_range) == 2):
        l, h = slider_range
    else :
        l, h = 0, 100;
    data_filtered = df[df['sleep_total'].between(l,h)]
    return  {
                'data': [
                    go.Scatter(
                        x=data_filtered[data_filtered['vore'] == i]['bodywt'] if i in input_value else [],
                        y=data_filtered[data_filtered['vore'] == i]['sleep_total'] if i in input_value else [],
                        text=df[df['vore'] == i]['name'],
                        mode='markers',
                        opacity=0.7,
                        marker={
                            'size': 15,
                            'line': {'width': 0.5, 'color': 'white'}
                        },
                        name=i
                    ) for i in df_vore
                ], ...
```


### Dash State

Dash include the reactive object `State` to keep informed of a callback function of some reactive information 
without the need to invoke events for it.

For example we add a button nearby our slide, like this
```python
        html.Div([
            html.Div(
                dcc.RangeSlider(
                    id='my-slider',
                    min= 0, max= 100, value=[0,100],
                    step= 0.1,
                ),
                style={
                    'width': '60%',
                    'display': 'inline-block',
                    'paddingLeft': '10%',
                    'paddingRight': '10%'
                }
            ),
            html.Button('Update filter', id='my-button')
        ],
            style={
                'marginTop': '5%',
                'marginBottom': '5%'
            }
        )
```

and we load the `State`object from the `dash.dependencies` library
```python
from dash.dependencies import Input, Output, State
```

then we modify the decorator and the definition of the callback function `update_output_graph` as follows
```python
@app.callback(
    [Output('my-graph', 'figure'),
     Output('my-box-plot', 'figure'),],
    [Input('my-multi-dropdown', 'value'),
     Input('my-button', 'n_clicks')],
    [State('my-slider', 'value')]
)
def update_output_graph(input_value, n_clicks, slider_range):
```

